In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-03-01 13:51:04,578 utils 345 [INFO]    [logger_func] start 


In [6]:
base = utils.read_df_pkl('../input/base_type_group.gz')[[key, target, 'clf_pred']].set_index(key)
path_list = glob.glob('../stack/0301*151feat*.gz')
for num, path in enumerate(path_list):
    tmp = utils.read_pkl_gzip(path).set_index(key)['prediction']
    if path.count('all'):
        col = 'all'
    elif path.count('lower'):
        col = 'low'
    elif path.count('upper'):
        col = 'high'
    base[col] = tmp
base.head()

100%|██████████| 1/1 [00:00<00:00,  3.91it/s]


,target,clf_pred,low,high,all
card_id,,,,,
C_ID_92a2005557,-0.820283,0.000444,-0.217902,NaN,-0.221895
C_ID_3d0044924f,0.392913,0.007831,0.157319,NaN,-0.048150
C_ID_d639edf6cd,0.688056,0.004074,0.537056,NaN,0.843324
C_ID_186d6a6901,0.142495,0.000797,0.236349,NaN,0.108849
C_ID_cdbd2c0db2,-0.159749,0.000251,-0.171607,NaN,-0.145486


In [22]:
pred_col = 'prediction'
all_ratio = 0.4
part_ratio = 0.6
base.loc[base['clf_pred']<=0.01, pred_col] =  base.loc[base['clf_pred']<=0.01, 'all'] * all_ratio + base.loc[base['clf_pred']<=0.01, 'low'] * part_ratio
base.loc[base['clf_pred']>0.01, pred_col] =  base.loc[base['clf_pred']>0.01, 'all'] * all_ratio + base.loc[base['clf_pred']>0.01, 'high'] * part_ratio
base.head()

,target,clf_pred,low,high,all,prediction
card_id,,,,,,
C_ID_92a2005557,-0.820283,0.000444,-0.217902,NaN,-0.221895,-0.219499
C_ID_3d0044924f,0.392913,0.007831,0.157319,NaN,-0.048150,0.075132
C_ID_d639edf6cd,0.688056,0.004074,0.537056,NaN,0.843324,0.659563
C_ID_186d6a6901,0.142495,0.000797,0.236349,NaN,0.108849,0.185349
C_ID_cdbd2c0db2,-0.159749,0.000251,-0.171607,NaN,-0.145486,-0.161159


In [23]:
submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred_col]
submit.to_csv(f'../submit/0301_evgency_solution_feat_go_feature_all{all_ratio}_part{part_ratio}.csv', index=True)